In [1]:
import tensorflow.keras as keras
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.0.0-alpha0'

In [3]:
tf.executing_eagerly()

True

In [4]:
batch_size = 32

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [5]:
from tensorflow.keras.models import load_model

In [6]:
from keras.datasets import cifar10
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator



batch_size = 32
num_classes = 10

num_predictions = 20


# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model = load_model('./combined.h5')
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])




Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


W0420 21:30:48.923893 140361278195520 hdf5_format.py:261] Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.


In [7]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 4s 406us/sample - loss: 0.7819 - accuracy: 0.8640
Test loss: 0.7818754182875156
Test accuracy: 0.864


In [8]:
conf = model.get_config()

In [9]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 8, 8, 256)       

In [10]:
for i, layer in enumerate(conf['layers']):
    if layer['class_name'] == 'Conv2D':
        print(i)

0
1
3
4
6
7
8
10
11
12
14
15


In [11]:
get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[14].output, model.layers[15].output])

In [12]:
get_output.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [13]:
x, y = get_output(x_test[0:1])

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

dataset = dataset.batch(batch_size)
dataset = dataset.repeat()

dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

dataset_test = dataset.batch(batch_size)
dataset_test = dataset.repeat()

In [15]:
get_output.output[0].shape

TensorShape([None, 2, 2, 512])

In [16]:
inputs = tf.keras.Input(shape=get_output.output[0].shape[1::])
X = tf.keras.layers.SeparableConv2D(filters=get_output.output[1].shape[-1], kernel_size= (3,3), padding='Same')(inputs)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.ReLU()(X)
X = tf.keras.layers.SeparableConv2D(filters=get_output.output[1].shape[-1], kernel_size=(3,3), padding='Same')(X)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.ReLU()(X)
replacement_layers = tf.keras.Model(inputs=inputs, outputs=X)

In [17]:
replacement_layers.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2, 2, 512)]       0         
_________________________________________________________________
separable_conv2d (SeparableC (None, 2, 2, 512)         267264    
_________________________________________________________________
batch_normalization_v2 (Batc (None, 2, 2, 512)         2048      
_________________________________________________________________
re_lu (ReLU)                 (None, 2, 2, 512)         0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 2, 2, 512)         267264    
_________________________________________________________________
batch_normalization_v2_1 (Ba (None, 2, 2, 512)         2048      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 2, 2, 512)         0   

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)

loss_object = tf.losses.MeanSquaredError()

In [19]:
replacement_layers.compile(loss=loss_object, optimizer=optimizer)

In [20]:
import math
class LayerBatch(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(50000 / 32)
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y
    
import math
class LayerTest(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(10000 / 32)
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y

In [21]:
save = tf.keras.callbacks.ModelCheckpoint('./replacement_layers_3_batch_relu.h5', verbose=1, save_weights_only=True, save_best_only=True)

In [22]:
train_gen = LayerBatch(get_output, dataset)
test_gen = LayerTest(get_output, dataset_test)

In [23]:
replacement_layers.fit_generator(generator=train_gen, epochs=50, validation_data=test_gen ,verbose=0, callbacks=[save])


Epoch 00001: val_loss improved from inf to 0.00061, saving model to ./replacement_layers_3_batch_relu.h5

Epoch 00002: val_loss improved from 0.00061 to 0.00029, saving model to ./replacement_layers_3_batch_relu.h5

Epoch 00003: val_loss did not improve from 0.00029

Epoch 00004: val_loss improved from 0.00029 to 0.00026, saving model to ./replacement_layers_3_batch_relu.h5

Epoch 00005: val_loss improved from 0.00026 to 0.00025, saving model to ./replacement_layers_3_batch_relu.h5

Epoch 00006: val_loss improved from 0.00025 to 0.00024, saving model to ./replacement_layers_3_batch_relu.h5

Epoch 00007: val_loss improved from 0.00024 to 0.00021, saving model to ./replacement_layers_3_batch_relu.h5

Epoch 00008: val_loss did not improve from 0.00021

Epoch 00009: val_loss improved from 0.00021 to 0.00021, saving model to ./replacement_layers_3_batch_relu.h5

Epoch 00010: val_loss improved from 0.00021 to 0.00020, saving model to ./replacement_layers_3_batch_relu.h5

Epoch 00011: val_lo

In [24]:
replacement_json = replacement_layers.to_json()
with open('replacement_layers_3_batch_relu.json', 'w') as json_file:
    json_file.write(replacement_json)

In [25]:
del replacement_layers

In [26]:
with open('replacement_layers_3_batch_relu.json', 'r') as json_file:
    replacement_layers = tf.keras.models.model_from_json(json_file.read())

replacement_layers.load_weights('replacement_layers_3_batch_relu.h5')

In [27]:
replacement_layers.compile(loss=loss_object, optimizer=optimizer)

In [28]:
replacement_layers.evaluate_generator(test_gen)

0.00011921932925648702

In [29]:
get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[14].output])

In [30]:
get_output.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [31]:
new_joint = tf.keras.Model(inputs=get_output.input, outputs=replacement_layers(get_output.output))

In [32]:
new_joint.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [33]:
bottom_half = tf.keras.Sequential()
for layer in model.layers[16::]:
    bottom_half.add(layer)

In [34]:
bottom_half.build(input_shape=new_joint.output.shape)


In [35]:
bottom_half.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_2 (Separabl (None, 2, 2, 512)         267264    
_________________________________________________________________
batch_normalization_v2_2 (Ba (None, 2, 2, 512)         2048      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 2, 2, 512)         0         
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 2, 2, 512)         267264    
_________________________________________________________________
batch_normalization_v2_3 (Ba (None, 2, 2, 512)         2048      
_________________________________________________________________
re_lu_3 (ReLU)               (None, 2, 2, 512)         0         
_________________________________________________________________
block5_pool (MaxPooling2D)   (None, 1, 1, 512)         0

In [36]:
combined = tf.keras.Model(inputs=new_joint.input, outputs=bottom_half(new_joint.output))

In [37]:
combined.layers[-1].trainable=False
combined.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [38]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 8, 8, 256)       

In [39]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 3s 295us/sample - loss: 0.7819 - accuracy: 0.8640
Test loss: 0.7818754182875156
Test accuracy: 0.864


In [40]:
combined.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [41]:
del bottom_half, new_joint, replacement_layers, model

In [42]:
scores = combined.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 3s 294us/sample - loss: 0.7708 - accuracy: 0.8623
Test loss: 0.7707621360957623
Test accuracy: 0.8623


In [43]:
new_combined = tf.keras.Sequential()
new_layers = []
new_combined.add(tf.keras.layers.Input(shape=(32,32,3)))
accum = 0
for layer in combined.layers:
    if hasattr(layer, 'layers'):
        for sublayer in layer.layers:
            if(sublayer.__class__.__name__ != 'InputLayer'): 
                new_layers.append((sublayer.__class__.__name__, sublayer.get_config(), accum))
            accum += 1
    else:
        
        new_layers.append((layer.__class__.__name__, layer.get_config(), accum))
        accum += 1 



In [44]:

for i, layer in enumerate(new_layers):
    new_combined.add(keras.layers.deserialize(
                            {'class_name': layer[0], 
                             'config': layer[1]}))

In [45]:
new_combined.build()

In [46]:
accum = 0
for i, layer in enumerate(combined.layers):
    if hasattr(layer, 'layers'):
        
        for sublayer in layer.layers:
            print(f'{accum} sub is {sublayer} new is {new_combined.layers[accum]}')
            if(sublayer.__class__.__name__ != 'InputLayer'):              
                new_combined.layers[accum].set_weights(sublayer.get_weights())
                accum += 1
#             else:
#                 accum += 1
        continue
    else:
        print(layer)
        new_combined.layers[accum].set_weights(layer.get_weights())
        accum +=1 



16 sub is <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fa7482446d8> new is <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x7fa7100e1d30>
16 sub is <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x7fa748249438> new is <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x7fa7100e1d30>
17 sub is <tensorflow.python.keras.layers.normalization.BatchNormalizationV2 object at 0x7fa72034a7b8> new is <tensorflow.python.keras.layers.normalization.BatchNormalizationV2 object at 0x7fa710133b70>
18 sub is <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fa748247080> new is <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fa710045a20>
19 sub is <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x7fa748247e10> new is <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x7fa710095860>
20 sub is <tensorflow.python.keras.layer

In [47]:
new_combined.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
del combined

In [48]:
new_save=tf.keras.callbacks.ModelCheckpoint('./refactor_finetune.h5', verbose=1, save_weights_only=False, save_best_only=True)

In [49]:
new_combined.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=3, callbacks=[new_save])

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
49952/50000 [============================>.] - ETA: 0s - loss: 0.0710 - accuracy: 0.9824
Epoch 00001: val_loss improved from inf to 1.00729, saving model to ./refactor_finetune.h5
50000/50000 [==============================] - 68s 1ms/sample - loss: 0.0709 - accuracy: 0.9824 - val_loss: 1.0073 - val_accuracy: 0.8622
Epoch 2/3
49952/50000 [============================>.] - ETA: 0s - loss: 0.0730 - accuracy: 0.9833
Epoch 00002: val_loss did not improve from 1.00729
50000/50000 [==============================] - 66s 1ms/sample - loss: 0.0730 - accuracy: 0.9833 - val_loss: 1.1739 - val_accuracy: 0.8641
Epoch 3/3
49984/50000 [============================>.] - ETA: 0s - loss: 0.0769 - accuracy: 0.9839
Epoch 00003: val_loss did not improve from 1.00729
50000/50000 [==============================] - 67s 1ms/sample - loss: 0.0769 - accuracy: 0.9839 - val_loss: 1.1311 - val_accuracy: 0.8553


In [50]:
new_combined.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         multiple                  0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)        